# GITM

In [ ]:
from kamodo.kamodo import Kamodo
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
init_notebook_mode(connected = True)
import plotly
import spacepy
import os

In [ ]:
from kamodo import readers

In [ ]:
from kamodo.readers.gitm import gitm as gitm
from kamodo.readers.gitm import gitm_alt_plots as gap
from kamodo.readers.gitm import gitm_plot_rout as gpr

In [ ]:
#datafile = '../../../data/GITMdefault/3DLST_t150317_180000.bin'
#datafile = '../../../data/GITMdefault/3DLST_t150318_120000.bin'
#datafile = '../../../data/GITMwSWMF/3DLST_t150317_180000.bin'
#datafile = '../../../data/GITMwSWMF/3DLST_t150318_120000.bin'

import glob
files = glob.glob('../../../data/IN/*.bin')
#files = glob.glob('../../../data/jasoon_shim_071418_IT_1/*.bin')
datafile = files[0]
datafile

In [ ]:
latkey = "dLat"
lonkey = "dLon"
altkey = "Altitude"
gData = gitm.GitmBin(datafile)

In [ ]:
gda = gData[altkey]
gda.attrs

import numpy as np

from kamodo.kamodo import Kamodo

lon = np.unique(gData[lonkey])
lat = np.unique(gData[latkey])
alt = np.unique(gData[altkey])
pos = np.random.random((30)).reshape(10, 3)

In [ ]:
sphere = Kamodo()
sphere['theta'] = lambda lat: np.pi*lat/180
sphere['phi'] = lambda lon: np.pi*lon/180
sphere['r'] = lambda alt: alt + 6.3781E6
sphere['x(lon,lat,alt)'] = 'r*cos(theta)*cos(phi)'
sphere['y(lon,lat,alt)'] = 'r*cos(theta)*sin(phi)'
sphere['z(alt,lat)'] = 'r*sin(theta)'
#sphere

In [ ]:
from scipy.interpolate import RegularGridInterpolator

def wrap_interpolator(lon, lat, alt, v):
    rgi = RegularGridInterpolator((lon, lat, alt), v, bounds_error = False)
    def wrapped(lon, lat, alt):
        return rgi((lon,lat,alt))
    return wrapped
    

In [ ]:
def kamodofy_names(name, name_maps):
    """replaces all substrings in name with those given by name_maps"""
    for old, new in name_maps:
        name = name.replace(old, new)
    return name

In [ ]:
kamodo = Kamodo(verbose = False)

for k,v in gData.items():
    var_name = kamodofy_names(k,[
            ('Vn (up) (m/s)', 'V_n__u__p [m/s]'),
            ('Vi (east) (m/s)', 'V_i__e__a__s__t [m/s]'),
            ('Vi (up) (m/s)', 'V_i__u__p [m/s]'),
            ('Vi (north) (m/s)', 'V_i__n__o__r__t__h [m/s]'),
            ('Vn (north) (m/s)', 'V_n__n__o__r__t__h [m/s]'),
            ('Vn (east) (m/s)', 'V_n__e__a__s__t [m/s]'),
            ('Rho (kg/m3)', 'rho (kg/m^3)'),
            ('O_4SP_!U+!N', 'O!U+_4SP_!N'),
            ('LT', 'lt [hours]'),
            ('V!Dn!N (up,N(!U4!NS)           )', 'V_N__u__pCOMMALEFT4SRIGHT [m/s]'),
            ('N(!U2!ND)','N__2D [kg/m^3]'),
            (" ", ''),
            ("O(!U3!NP)", "OLEFT3PRIGHT"),
            ('(','['),
            (')',']'),
            ('!D', '_'),
            ('!N',''),
            ('[/m3]','[1/m^3]'),
            ('!U','__'),
            ('+', 'plus'),
            ('e-', 'eminus'),
        ])
    if k not in ['time']:
        try:
            kamodo[var_name] = wrap_interpolator(lon, lat, alt, v)
        except:
            print (k)
            raise


from kamodo.kamodo import kamodofy

# define lat, lon, an alt in terms of r, theta, phi with defaults that correspond to the model
@kamodofy(units = 'degrees')
def LAT(theta = sphere.theta(lat[1:-1])):
    return 180*theta/np.pi


@kamodofy(units = 'degrees')
def LON(phi = sphere.phi(lon[1:-1])):
    return 180*phi/np.pi+180


@kamodofy(units = 'm')
def ALT(r = sphere.r(alt[1:-1])):
    '''must generate altitude in meters'''
    return r - 6.3781E6

kamodo['ALT'] = ALT
kamodo['LON'] = LON
kamodo['LAT'] = LAT

kamodo['r'] = 'sqrt(x**2 + y**2 + z**2)'
kamodo['theta'] = 'asin(z/r)' #colatitude
kamodo['phi'] = 'atan2(y,x)'
#kamodo


In [ ]:
newvars = ["rho", "NeutralTemperature", "V_n__u__p", "V_i__e__a__s__t", "NO", "N_2", "NO__plus", "OLEFT3PRIGHT", "O__plus_4SP_", "eminus", "V_i__u__p", "N_2__plus", "V_i__n__o__r__t__h", "V_n__e__a__s__t", "O_2__plus", "O_2", "V_n__n__o__r__t__h"]
#newvars

In [ ]:
ilat = np.linspace(-90, 90, 91)
ilon = np.linspace(0, 360, 181)
ialt = np.array([400000.])

In [ ]:
# Have to make lat into a column
def rhoK(ilon, ilat, ialt):
    return kamodo.rho(ilon, ilat.reshape(-1,1), ialt)
def NeutralTemperatureK(ilon, ilat, ialt):
    return kamodo.NeutralTemperature(ilon, ilat.reshape(-1,1), ialt)
def V_n__u__pK(ilon, ilat, ialt):
    return kamodo.V_n__u__p(ilon, ilat.reshape(-1,1), ialt)
def V_i__e__a__s__tK(ilon, ilat, ialt):
    return kamodo.V_i__e__a__s__t(ilon, ilat.reshape(-1,1), ialt)
def NOK(ilon, ilat, ialt):
    return kamodo.NO(ilon, ilat.reshape(-1,1), ialt)
def N_2K(ilon, ilat, ialt):
    return kamodo.N_2(ilon, ilat.reshape(-1,1), ialt)
def NO__plusK(ilon, ilat, ialt):
    return kamodo.NO__plus(ilon, ilat.reshape(-1,1), ialt)
def OLEFT3PRIGHTK(ilon, ilat, ialt):
    return kamodo.OLEFT3PRIGHT(ilon, ilat.reshape(-1,1), ialt)
def O__plus_4SP_K(ilon, ilat, ialt):
    return kamodo.O__plus_4SP_(ilon, ilat.reshape(-1,1), ialt)
def eminusK(ilon, ilat, ialt):
    return kamodo.eminus(ilon, ilat.reshape(-1,1), ialt)
def V_i__u__pK(ilon, ilat, ialt):
    return kamodo.V_i__u__p(ilon, ilat.reshape(-1,1), ialt)
def N_2__plusK(ilon, ilat, ialt):
    return kamodo.N_2__plus(ilon, ilat.reshape(-1,1), ialt)
def V_i__n__o__r__t__hK(ilon, ilat, ialt):
    return kamodo.V_i__n__o__r__t__h(ilon, ilat.reshape(-1,1), ialt)
def V_n__e__a__s__tK(ilon, ilat, ialt):
    return kamodo.V_n__e__a__s__t(ilon, ilat.reshape(-1,1), ialt)
def O_2__plusK(ilon, ilat, ialt):
    return kamodo.O_2__plus(ilon, ilat.reshape(-1,1), ialt)
def O_2K(ilon, ilat, ialt):
    return kamodo.O_2(ilon, ilat.reshape(-1,1), ialt)
def V_n__n__o__r__t__hK(ilon, ilat, ialt):
    return kamodo.V_n__n__o__r__t__h(ilon, ilat.reshape(-1,1), ialt)

In [ ]:
kamodo['rhoK(ilon, ilat, ialt)'] = rhoK
kamodo['NeutralTemperatureK(ilon, ilat, ialt)'] = NeutralTemperatureK
kamodo['V_n__u__pK(ilon, ilat, ialt)'] = V_n__u__pK
kamodo['V_i__e__a__s__tK(ilon, ilat, ialt)'] = V_i__e__a__s__tK
kamodo['NOK(ilon, ilat, ialt)'] = NOK
kamodo['N_2K(ilon, ilat, ialt)'] = N_2K
kamodo['NO__plusK(ilon, ilat, ialt)'] = NO__plusK
kamodo['OLEFT3PRIGHTK(ilon, ilat, ialt)'] = OLEFT3PRIGHTK
kamodo['O__plus_4SP_K(ilon, ilat, ialt)'] = O__plus_4SP_K
kamodo['eminusK(ilon, ilat, ialt)'] = eminusK
kamodo['V_i__u__pK(ilon, ilat, ialt)'] = V_i__u__pK
kamodo['N_2__plusK(ilon, ilat, ialt)'] = N_2__plusK
kamodo['V_i__n__o__r__t__hK(ilon, ilat, ialt)'] = V_i__n__o__r__t__hK
kamodo['V_n__e__a__s__tK(ilon, ilat, ialt)'] = V_n__e__a__s__tK
kamodo['O_2__plusK(ilon, ilat, ialt)'] = O_2__plusK
kamodo['O_2K(ilon, ilat, ialt)'] = O_2K
kamodo['V_n__n__o__r__t__hK(ilon, ilat, ialt)'] = V_n__n__o__r__t__hK

In [ ]:
resultsALL = []

results = kamodo.evaluate('rhoK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['rhoK'])

results = kamodo.evaluate('NeutralTemperatureK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['NeutralTemperatureK'])

results = kamodo.evaluate('V_n__u__pK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['V_n__u__pK'])

results = kamodo.evaluate('V_i__e__a__s__tK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['V_i__e__a__s__tK'])

results = kamodo.evaluate('NOK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['NOK'])

results = kamodo.evaluate('N_2K', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['N_2K'])

results = kamodo.evaluate('NO__plusK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['NO__plusK'])

results = kamodo.evaluate('OLEFT3PRIGHTK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['OLEFT3PRIGHTK'])

results = kamodo.evaluate('O__plus_4SP_K', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['O__plus_4SP_K'])

results = kamodo.evaluate('eminusK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['eminusK'])

results = kamodo.evaluate('V_i__u__pK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['V_i__u__pK'])

results = kamodo.evaluate('N_2__plusK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['N_2__plusK'])

results = kamodo.evaluate('V_i__n__o__r__t__hK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['V_i__n__o__r__t__hK'])

results = kamodo.evaluate('V_n__e__a__s__tK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['V_n__e__a__s__tK'])

results = kamodo.evaluate('O_2__plusK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['O_2__plusK'])

results = kamodo.evaluate('O_2K', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['O_2K'])

results = kamodo.evaluate('V_n__n__o__r__t__hK', ilon = ilon, ilat = ilat, ialt=ialt)
resultsALL.append(results['V_n__n__o__r__t__hK'])

In [ ]:
#len(resultsALL)
#resultsALL

In [ ]:
head, tail = os.path.split(datafile)
idx1 = tail.find('_')
idx2 = tail.find('.')
newfile = '400km' + tail[idx1:idx2] + '.dat'
outF = open(newfile, "w")
header="#GITM 2x2 degree grid extracted at 400km\n#run = nnn\n#datafile = " + tail + "\nLatitude Longitude " + " ".join("%s" % n for n in newvars) + "\n"
outF.write(header)
i = 0
while i < len(ilat):
    latv = ilat[i]
    j = 0
    while j < len(ilon):
        lonv = ilon[j]
        vv = []
        n = 0
        while n < len(newvars):
            vv.append(resultsALL[n][i][j])
            n += 1
        line = '{:.0f} {:.0f} '.format(latv, lonv) + " ".join("%.5e" % n for n in vv) + "\n"
        outF.write(line)
        j += 1
    i += 1

outF.close()

# Plotting and other misc stuff ...

In [ ]:
# 3D plot of density at 400km altitude
iplot(kamodo.plot(rhoK = dict(ilon = ilon, ilat = ilat, ialt = ialt)))

In [ ]:
def rhoK2D(ilon, ilat):
    return kamodo.rho(ilon, ilat.reshape(-1,1), ialt)

kamodo['rhoK2D(ilon, ilat)'] = rhoK2D

In [ ]:
# 2D plot of density at 400km altitude
iplot(kamodo.plot(rhoK2D = dict(ilon = ilon, ilat = ilat)))

In [ ]:
# 3D plot of density at 400km altitude in spherical coords
kamodo['Srho'] = 'rho(LON, LAT, ALT)'
alt_slice = 400000.
llon, llat = np.meshgrid(np.array(lon), np.array(lat))
iplot(kamodo.plot(Srho = dict(
        x = sphere.x(llon, llat, alt_slice),
        y = sphere.y(llon, llat, alt_slice),
        z = sphere.z(alt_slice, llat))))

In [ ]:
results = kamodo.evaluate('rhoK2D', ilon = ilon, ilat = ilat)
results['rhoK2D']
results

In [ ]:
for k,v in gData.items():
    print(k)

In [ ]:
print (gData['Rho (kg/m3)'].attrs)
#print gData.attrs

## Plot a dummy trajectory over the plot

In [ ]:
def trajectory(t = np.linspace(0, 360, 60)):
    lon = t
    lat = 50*np.sin(2*np.pi*t/360)
    return np.vstack((lon, lat)).T

In [ ]:
kamodo['sat'] = trajectory

In [ ]:
iplot(kamodo.plot( 'sat'))

In [ ]:
iplot(kamodo.plot('sat', rhoK2D = dict(ilon = ilon, ilat = ilat)))

## Extract values on trajectory
- currently reading an ASCII DMSP file
- does not handle periodicity well (convert to XYZ from LAT/LON to show better)
- timestamp in file is number of seconds in the day

In [ ]:
import pandas as pd
import glob

In [ ]:
# This reads in a DMSP orbit file to get positions
files = glob.glob('../../../data/f13*')
datafile = files[0]
datafile

In [ ]:
df = pd.read_csv(datafile, delimiter= '\s+', index_col=False)
df

In [ ]:
# Defaults to row basis, so transpose it to get the columns I want as a numpy array
import time
import datetime

new=(df.to_numpy()).transpose()
tmpDate=new[0]
tmpTime=new[1]
dAlt=new[4]
dGLAT=new[5]
dGLONG=new[6]
dTime = []
for i in range(len(tmpTime)):
    d = tmpDate[0]
    y= int(str(d)[0:3])+1900
    doy = int(str(d)[3:6])
    d = str(datetime.datetime(y, 1, 1) + datetime.timedelta(doy - 1))[0:10]
    t = time.strftime('%H:%M:%S', time.gmtime(tmpTime[i]))
    dTime.append(d + ' ' + t)

print(tmpDate[0],tmpTime[0],' ==>',dTime[0],'\n',dAlt,'\n',dGLAT,'\n',dGLONG)

In [ ]:
# Force altitude to 400km as actual DMSP position is outside of simulation
dRho = []
for i in range(len(dAlt)):
    tmplon = np.array([dGLONG[i]])
    tmplat = np.array([dGLAT[i]])
    tmpalt = np.array([400000.])
    results = kamodo.evaluate('rhoK', ilon = tmplon, ilat = tmplat, ialt=tmpalt)
    newvalue = results['rhoK'][0][0]
    dRho.append(newvalue)

In [ ]:
def DMSPtrajectory1(rho = dRho):
    lon = dGLONG
    lat = dGLAT
    return np.vstack((lon, lat)).T

In [ ]:
def DMSPtrajectory2(rho = dRho):
    lon = dGLONG
    lat = dGLAT
    time = dTime
    return np.vstack((time, rho)).T

In [ ]:
kamodo['DMSP1'] = DMSPtrajectory1

In [ ]:
kamodo['DMSP2'] = DMSPtrajectory2

In [ ]:
iplot(kamodo.plot( 'DMSP2' ))

In [ ]:
iplot(kamodo.plot( 'DMSP1', rhoK2D = dict(ilon = ilon, ilat = ilat) ))

In [ ]:
dPOS = np.array([dGLONG,dGLAT,dAlt])
dPOS